<a href="https://colab.research.google.com/github/Wlnfadhil/Analisa-Data-Air-Quality-Control/blob/main/submission/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyek Analisis Data: PRSA_Data_20130301-20170228
- **Nama:** Wildan Fadhil Nazaruddin
- **Email:** wildanfadhil76@gmial.com
- **ID Dicoding:**

## Air  Data Quality

Air particulate matter in various regions has a significant impact on public health. Year-over-year data analysis is crucial in guiding informed decision-making to mitigate the effects of global warming. This project aims to analyze weather conditions and air quality across different regions of China to better understand environmental trends and their implications. The dataset used in this project includes data from 12 provinces in China, which will be examined to identify patterns and provide insights for more effective policy-making in maintaining air quality and public health. This analysis can also contribute to developing better strategies for mitigating the ongoing climate change.

### 1.1 Clases

- PRSA_Data_Aotizhongxin: Data collected from the Aotizhongxin station.
- PRSA_Data_Changping: Data from the Changping station.
- PRSA_Data_Dingling: Data from the Dingling station.
- PRSA_Data_Dongsi: Data from the Dongsi station.
- PRSA_Data_Guanyuan: Data from the Guanyuan station.
- PRSA_Data_Gucheng: Data from the Gucheng station.
- PRSA_Data_Huairou: Data from the Huairou station.
- PRSA_Data_Tiantan: Data from the Tiantan station.
- PRSA_Data_Wanliu: Data from the Wanliu station.

### 1.2 Methodology

1. Data Collection and Cleaning:
- - -
First, we will consolidate the datasets from the 12 provinces. Data cleaning will involve handling missing values, correcting inconsistencies, and ensuring all datasets are standardized.
Descriptive Statistics:
- - -
Descriptive statistics such as mean, median, standard deviation, and interquartile range (IQR) will be used to summarize the key characteristics of the air particulate data (PM2.5, PM10) across different provinces. Visualizations like histograms, box plots, and time series plots will be used to better understand the distribution and spread of the data.
Correlation Analysis:
- - -
To identify the relationship between different pollutants and weather conditions, we will conduct a Pearson or Spearman correlation analysis. This will help in understanding how temperature, humidity, or wind speed affect the levels of particulate matter in the air.
Trend Analysis:
- - -
Trend analysis will be performed to observe how air quality changes over time (seasonally or annually) and across provinces. We will use time series decomposition to break down the data into trend, seasonal, and residual components, enabling a clearer view of underlying patterns.
Geospatial Analysis:
- - -
By plotting data on maps, we will explore the geographical distribution of air particulate matter across provinces, using spatial visualization tools to observe how air quality varies between regions.
Hypothesis Testing:
- - -
Statistical hypothesis tests (such as t-tests or ANOVA) will be used to compare air quality between different regions or time periods, determining whether observed differences are statistically significant.

### 1.3 Deployment

Data set

```
! git clone https://github.com/Wlnfadhil/Analisa-Data-Air-Quality-Control.git
```



## 1 code enggine

In [1]:
! git clone https://github.com/Wlnfadhil/Analisa-Data-Air-Quality-Control.git

Cloning into 'Analisa-Data-Air-Quality-Control'...
remote: Enumerating objects: 15574, done.
remote: Counting objects: 100% (1698/1698), done.
remote: Compressing objects: 100% (1619/1619), done.
remote: Total 15574 (delta 68), reused 1681 (delta 63), pack-reused 13876 (from 1)
Receiving objects: 100% (15574/15574), 102.17 MiB | 6.72 MiB/s, done.
Resolving deltas: 100% (1155/1155), done.
Updating files: 100% (15062/15062), done.


## 2 Menentukan Pertanyaan Bisnis

- Pertanyaan 1 :
  - What are the primary trends in air quality levels (PM2.5, PM10) across the 12 provinces in China over the observed time period (2013-2017)?
- Pertanyaan 2 :
  - How do various weather conditions (e.g., temperature, humidity, wind speed) correlate with particulate matter concentrations (PM2.5, PM10) in each province?
- Pertanyaan 3 :
   - Which regions show the highest and lowest levels of air particulate matter, and what factors contribute to these regional differences?
- Pertanyaan 4 :   
   - How do seasonal variations (e.g., winter vs. summer) impact air quality across the provinces, and what are the contributing factors?
- Pertanyaan 5 :
   - What actionable insights can be drawn from this analysis to inform policy decisions aimed at improving air quality and mitigating public health risks?


## 3 Import Semua Packages/Library yang Digunakan

In [2]:
!pip install pandasql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26772 sha256=dd6e1b77bf3ddc146c13964c927159cfaccce43814df54c659ef6d945341aa74
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from IPython.display import Markdown, display
from pandasql import sqldf


## 4 Data Wrangling

### 4.1 Gathering Data

#### 4.1.1 Load data

##### 4.1.1.1 Menentukan Direktori yang di tuju

In [4]:
# Mendefinisikan direktori saat ini
current_dir = os.getcwd()

In [5]:
# Daftar nama file CSV yang ada
csv_files = [
    "PRSA_Data_Aotizhongxin_20130301-20170228.csv",
    "PRSA_Data_Changping_20130301-20170228.csv",
    "PRSA_Data_Dingling_20130301-20170228.csv",
    "PRSA_Data_Dongsi_20130301-20170228.csv",
    "PRSA_Data_Guanyuan_20130301-20170228.csv",
    "PRSA_Data_Gucheng_20130301-20170228.csv",
    "PRSA_Data_Huairou_20130301-20170228.csv",
    "PRSA_Data_Nongzhanguan_20130301-20170228.csv",
    "PRSA_Data_Shunyi_20130301-20170228.csv",
    "PRSA_Data_Tiantan_20130301-20170228.csv",
    "PRSA_Data_Wanliu_20130301-20170228.csv",
    "PRSA_Data_Wanshouxigong_20130301-20170228.csv"
]


In [6]:
# Membaca setiap file CSV dan menyimpannya dalam DataFrame
dataframes = {}

In [7]:
for csv_file in csv_files:
    # Mendapatkan nama lokasi (misalnya, Aotizhongxin, Changping, dll.) dari nama file
    location = csv_file.split('_')[2]

    # Menggabungkan path untuk setiap file
    file_path = os.path.join(current_dir, "./Analisa-Data-Air-Quality-Control/submission/data", csv_file)

    # Membaca CSV ke dalam DataFrame
    df = pd.read_csv(file_path)

    # Menyimpan DataFrame ke dalam dictionary dengan kunci nama lokasi
    dataframes[location] = df

In [8]:
aotizhongxin_df = dataframes['Aotizhongxin']

changping_df = dataframes['Changping']

dingling_df = dataframes['Dingling']

dongsi_df = dataframes['Dongsi']

guanyuan_df = dataframes['Guanyuan']

gucheng_df = dataframes['Gucheng']

huairou_df = dataframes ['Huairou']

Nongzhanguan_df = dataframes['Nongzhanguan']

shunyi_df = dataframes['Shunyi']

tiantian_df = dataframes['Tiantan']

wanliu_df = dataframes['Wanliu']

Wanshouxigong_df = dataframes['Wanshouxigong']


##### 4.1.1.2 Melihat Info Data

###### 4.1.1.2.1 Aotizhongxin

In [ ]:
aotizhongxin_df.head(5)

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin


In [ ]:
aotizhongxin_df.describe()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM
count,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,34139.000000,34346.000000,34129.000000,34041.000000,33288.000000,33345.000000,35044.000000,35044.000000,35044.000000,35044.000000,35050.000000
mean,17532.500000,2014.662560,6.522930,15.729637,11.500000,82.773611,110.060391,17.375901,59.305833,1262.945145,56.353358,13.584607,1011.846920,3.123062,0.067421,1.708496
std,10122.249256,1.177213,3.448752,8.800218,6.922285,82.135694,95.223005,22.823017,37.116200,1221.436236,57.916327,11.399097,10.404047,13.688896,0.910056,1.204071
min,1.000000,2013.000000,1.000000,1.000000,0.000000,3.000000,2.000000,0.285600,2.000000,100.000000,0.214200,-16.800000,985.900000,-35.300000,0.000000,0.000000
25%,8766.750000,2014.000000,4.000000,8.000000,5.750000,22.000000,38.000000,3.000000,30.000000,500.000000,8.000000,3.100000,1003.300000,-8.100000,0.000000,0.900000
50%,17532.500000,2015.000000,7.000000,16.000000,11.500000,58.000000,87.000000,9.000000,53.000000,900.000000,42.000000,14.500000,1011.400000,3.800000,0.000000,1.400000
75%,26298.250000,2016.000000,10.000000,23.000000,17.250000,114.000000,155.000000,21.000000,82.000000,1500.000000,82.000000,23.300000,1020.100000,15.600000,0.000000,2.200000
max,35064.000000,2017.000000,12.000000,31.000000,23.000000,898.000000,984.000000,341.000000,290.000000,10000.000000,423.000000,40.500000,1042.000000,28.500000,72.500000,11.200000


###### 4.1.1.2.2 changping

In [ ]:
changping_df.head(5)

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,3.0,6.0,13.0,7.0,300.0,85.0,-2.3,1020.8,-19.7,0.0,E,0.5,Changping
1,2,2013,3,1,1,3.0,3.0,6.0,6.0,300.0,85.0,-2.5,1021.3,-19.0,0.0,ENE,0.7,Changping
2,3,2013,3,1,2,3.0,3.0,22.0,13.0,400.0,74.0,-3.0,1021.3,-19.9,0.0,ENE,0.2,Changping
3,4,2013,3,1,3,3.0,6.0,12.0,8.0,300.0,81.0,-3.6,1021.8,-19.1,0.0,NNE,1.0,Changping
4,5,2013,3,1,4,3.0,3.0,14.0,8.0,300.0,81.0,-3.5,1022.3,-19.4,0.0,N,2.1,Changping


In [ ]:
dingling_df.head(5)

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,4.0,4.0,3.0,NaN,200.0,82.0,-2.3,1020.8,-19.7,0.0,E,0.5,Dingling
1,2,2013,3,1,1,7.0,7.0,3.0,NaN,200.0,80.0,-2.5,1021.3,-19.0,0.0,ENE,0.7,Dingling
2,3,2013,3,1,2,5.0,5.0,3.0,2.0,200.0,79.0,-3.0,1021.3,-19.9,0.0,ENE,0.2,Dingling
3,4,2013,3,1,3,6.0,6.0,3.0,NaN,200.0,79.0,-3.6,1021.8,-19.1,0.0,NNE,1.0,Dingling
4,5,2013,3,1,4,5.0,5.0,3.0,NaN,200.0,81.0,-3.5,1022.3,-19.4,0.0,N,2.1,Dingling


In [ ]:
dongsi_df.head(5)

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,9.0,9.0,3.0,17.0,300.0,89.0,-0.5,1024.5,-21.4,0.0,NNW,5.7,Dongsi
1,2,2013,3,1,1,4.0,4.0,3.0,16.0,300.0,88.0,-0.7,1025.1,-22.1,0.0,NW,3.9,Dongsi
2,3,2013,3,1,2,7.0,7.0,NaN,17.0,300.0,60.0,-1.2,1025.3,-24.6,0.0,NNW,5.3,Dongsi
3,4,2013,3,1,3,3.0,3.0,5.0,18.0,NaN,NaN,-1.4,1026.2,-25.5,0.0,N,4.9,Dongsi
4,5,2013,3,1,4,3.0,3.0,7.0,NaN,200.0,84.0,-1.9,1027.1,-24.5,0.0,NNW,3.2,Dongsi


In [ ]:
guanyuan_df.head(5)

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,4.0,4.0,14.0,20.0,300.0,69.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Guanyuan
1,2,2013,3,1,1,4.0,4.0,13.0,17.0,300.0,72.0,-1.1,1023.2,-18.2,0.0,N,4.7,Guanyuan
2,3,2013,3,1,2,3.0,3.0,10.0,19.0,300.0,69.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Guanyuan
3,4,2013,3,1,3,3.0,6.0,7.0,24.0,400.0,62.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Guanyuan
4,5,2013,3,1,4,3.0,6.0,5.0,14.0,400.0,71.0,-2.0,1025.2,-19.5,0.0,N,2.0,Guanyuan


In [ ]:
guanyuan_df.head(5)

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,4.0,4.0,14.0,20.0,300.0,69.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Guanyuan
1,2,2013,3,1,1,4.0,4.0,13.0,17.0,300.0,72.0,-1.1,1023.2,-18.2,0.0,N,4.7,Guanyuan
2,3,2013,3,1,2,3.0,3.0,10.0,19.0,300.0,69.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Guanyuan
3,4,2013,3,1,3,3.0,6.0,7.0,24.0,400.0,62.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Guanyuan
4,5,2013,3,1,4,3.0,6.0,5.0,14.0,400.0,71.0,-2.0,1025.2,-19.5,0.0,N,2.0,Guanyuan


In [ ]:
huairou_df.head(5)

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,7.0,7.0,3.0,2.0,100.0,91.0,-2.3,1020.3,-20.7,0.0,WNW,3.1,Huairou
1,2,2013,3,1,1,4.0,4.0,3.0,NaN,100.0,92.0,-2.7,1020.8,-20.5,0.0,NNW,1.5,Huairou
2,3,2013,3,1,2,4.0,4.0,NaN,NaN,100.0,91.0,-3.2,1020.6,-21.4,0.0,NW,1.8,Huairou
3,4,2013,3,1,3,3.0,3.0,3.0,2.0,NaN,NaN,-3.3,1021.3,-23.7,0.0,NNW,2.4,Huairou
4,5,2013,3,1,4,3.0,3.0,7.0,NaN,300.0,86.0,-4.1,1022.1,-22.7,0.0,NNW,2.2,Huairou


In [ ]:
Nongzhanguan_df.head(5)

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,5.0,14.0,4.0,12.0,200.0,85.0,-0.5,1024.5,-21.4,0.0,NNW,5.7,Nongzhanguan
1,2,2013,3,1,1,8.0,12.0,6.0,14.0,200.0,84.0,-0.7,1025.1,-22.1,0.0,NW,3.9,Nongzhanguan
2,3,2013,3,1,2,3.0,6.0,5.0,14.0,200.0,83.0,-1.2,1025.3,-24.6,0.0,NNW,5.3,Nongzhanguan
3,4,2013,3,1,3,5.0,5.0,5.0,14.0,200.0,84.0,-1.4,1026.2,-25.5,0.0,N,4.9,Nongzhanguan
4,5,2013,3,1,4,5.0,5.0,6.0,21.0,200.0,77.0,-1.9,1027.1,-24.5,0.0,NNW,3.2,Nongzhanguan


In [ ]:
shunyi_df.head(5)

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,3.0,6.0,3.0,8.0,300.0,44.0,-0.9,1025.8,-20.5,0.0,NW,9.3,Shunyi
1,2,2013,3,1,1,12.0,12.0,3.0,7.0,300.0,47.0,-1.1,1026.1,-21.3,0.0,NW,9.4,Shunyi
2,3,2013,3,1,2,14.0,14.0,NaN,7.0,200.0,22.0,-1.7,1026.2,-23.0,0.0,NW,8.6,Shunyi
3,4,2013,3,1,3,12.0,12.0,3.0,5.0,NaN,NaN,-2.1,1027.3,-23.3,0.0,NW,6.6,Shunyi
4,5,2013,3,1,4,12.0,12.0,3.0,NaN,200.0,11.0,-2.4,1027.7,-22.9,0.0,NW,4.5,Shunyi


In [ ]:
tiantian_df.head(5)

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,6.0,6.0,4.0,8.0,300.0,81.0,-0.5,1024.5,-21.4,0.0,NNW,5.7,Tiantan
1,2,2013,3,1,1,6.0,29.0,5.0,9.0,300.0,80.0,-0.7,1025.1,-22.1,0.0,NW,3.9,Tiantan
2,3,2013,3,1,2,6.0,6.0,4.0,12.0,300.0,75.0,-1.2,1025.3,-24.6,0.0,NNW,5.3,Tiantan
3,4,2013,3,1,3,6.0,6.0,4.0,12.0,300.0,74.0,-1.4,1026.2,-25.5,0.0,N,4.9,Tiantan
4,5,2013,3,1,4,5.0,5.0,7.0,15.0,400.0,70.0,-1.9,1027.1,-24.5,0.0,NNW,3.2,Tiantan


In [ ]:
wanliu_df.head(5)

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,8.0,8.0,6.0,28.0,400.0,52.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Wanliu
1,2,2013,3,1,1,9.0,9.0,6.0,28.0,400.0,50.0,-1.1,1023.2,-18.2,0.0,N,4.7,Wanliu
2,3,2013,3,1,2,3.0,6.0,NaN,19.0,400.0,55.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Wanliu
3,4,2013,3,1,3,11.0,30.0,8.0,14.0,NaN,NaN,-1.4,1024.5,-19.4,0.0,NW,3.1,Wanliu
4,5,2013,3,1,4,3.0,13.0,9.0,NaN,300.0,54.0,-2.0,1025.2,-19.5,0.0,N,2.0,Wanliu


In [ ]:
Wanshouxigong_df.head(5)

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,9.0,9.0,6.0,17.0,200.0,62.0,0.3,1021.9,-19.0,0.0,WNW,2.0,Wanshouxigong
1,2,2013,3,1,1,11.0,11.0,7.0,14.0,200.0,66.0,-0.1,1022.4,-19.3,0.0,WNW,4.4,Wanshouxigong
2,3,2013,3,1,2,8.0,8.0,NaN,16.0,200.0,59.0,-0.6,1022.6,-19.7,0.0,WNW,4.7,Wanshouxigong
3,4,2013,3,1,3,8.0,8.0,3.0,16.0,NaN,NaN,-0.7,1023.5,-20.9,0.0,NW,2.6,Wanshouxigong
4,5,2013,3,1,4,8.0,8.0,3.0,NaN,300.0,36.0,-0.9,1024.1,-21.7,0.0,WNW,2.5,Wanshouxigong


## 5 Assessing Data

### 5.1 aotizhongxin

In [ ]:
aotizhongxin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 18 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   No       35064 non-null  int64  
 1   year     35064 non-null  int64  
 2   month    35064 non-null  int64  
 3   day      35064 non-null  int64  
 4   hour     35064 non-null  int64  
 5   PM2.5    34139 non-null  float64
 6   PM10     34346 non-null  float64
 7   SO2      34129 non-null  float64
 8   NO2      34041 non-null  float64
 9   CO       33288 non-null  float64
 10  O3       33345 non-null  float64
 11  TEMP     35044 non-null  float64
 12  PRES     35044 non-null  float64
 13  DEWP     35044 non-null  float64
 14  RAIN     35044 non-null  float64
 15  wd       34983 non-null  object 
 16  WSPM     35050 non-null  float64
 17  station  35064 non-null  object 
dtypes: float64(11), int64(5), object(2)
memory usage: 4.8+ MB


In [ ]:
aotizhongxin_df.isnull().sum()

No            0
year          0
month         0
day           0
hour          0
PM2.5       925
PM10        718
SO2         935
NO2        1023
CO         1776
O3         1719
TEMP         20
PRES         20
DEWP         20
RAIN         20
wd           81
WSPM         14
station       0
dtype: int64

In [ ]:
changping_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 18 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   No       35064 non-null  int64  
 1   year     35064 non-null  int64  
 2   month    35064 non-null  int64  
 3   day      35064 non-null  int64  
 4   hour     35064 non-null  int64  
 5   PM2.5    34290 non-null  float64
 6   PM10     34482 non-null  float64
 7   SO2      34436 non-null  float64
 8   NO2      34397 non-null  float64
 9   CO       33543 non-null  float64
 10  O3       34460 non-null  float64
 11  TEMP     35011 non-null  float64
 12  PRES     35014 non-null  float64
 13  DEWP     35011 non-null  float64
 14  RAIN     35013 non-null  float64
 15  wd       34924 non-null  object 
 16  WSPM     35021 non-null  float64
 17  station  35064 non-null  object 
dtypes: float64(11), int64(5), object(2)
memory usage: 4.8+ MB


In [ ]:
aotizhongxin_df.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
35059    False
35060    False
35061    False
35062    False
35063    False
Length: 35064, dtype: bool

In [ ]:

def cek_nilai_tidak_akurat(df):
    print("Pemeriksaan nilai tidak akurat:")

    # Definisikan rentang nilai yang dianggap akurat untuk setiap kolom
    rentang_akurat = {
        'PM2.5': (0, 1000),
        'PM10': (0, 1500),
        'SO2': (0, 2000),
        'NO2': (0, 2000),
        'CO': (0, 200),
        'O3': (0, 500),
        'TEMP': (-50, 50),
        'PRES': (800, 1100),
        'DEWP': (-50, 50),
        'RAIN': (0, 500),
        'WSPM': (0, 100)
    }

    hasil_tidak_akurat = {}

    for kolom, (batas_bawah, batas_atas) in rentang_akurat.items():
        if kolom in df.columns:
            nilai_tidak_akurat = df[(df[kolom] < batas_bawah) | (df[kolom] > batas_atas)]
            if not nilai_tidak_akurat.empty:
                hasil_tidak_akurat[kolom] = nilai_tidak_akurat[[kolom]]
                print(f"\nNilai tidak akurat ditemukan di kolom {kolom}:")
                print(nilai_tidak_akurat[[kolom]])

    if not hasil_tidak_akurat:
        print("Tidak ditemukan nilai yang tidak akurat dalam rentang yang ditentukan.")

    return hasil_tidak_akurat

# Menjalankan fungsi
hasil_tidak_akurat = cek_nilai_tidak_akurat(aotizhongxin_df)

Pemeriksaan nilai tidak akurat:

Nilai tidak akurat ditemukan di kolom CO:
          CO
0      300.0
1      300.0
2      300.0
3      300.0
4      300.0
...      ...
35059  400.0
35060  500.0
35061  700.0
35062  700.0
35063  600.0

[31776 rows x 1 columns]


In [ ]:

def cek_nilai_tidak_konsisten(df):
    print("Pemeriksaan nilai tidak konsisten:")

    hasil_tidak_konsisten = {}

    # 1. Memeriksa inkonsistensi antara PM2.5 dan PM10
    if 'PM2.5' in df.columns and 'PM10' in df.columns:
        pm_tidak_konsisten = df[df['PM2.5'] > df['PM10']]
        if not pm_tidak_konsisten.empty:
            hasil_tidak_konsisten['PM2.5 > PM10'] = pm_tidak_konsisten[['PM2.5', 'PM10']]
            print("\nDitemukan nilai PM2.5 yang lebih besar dari PM10:")
            print(pm_tidak_konsisten[['PM2.5', 'PM10']])

    # 2. Memeriksa inkonsistensi dalam arah angin
    if 'wd' in df.columns:
        arah_angin_valid = ['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE',
                            'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW']
        arah_angin_tidak_valid = df[~df['wd'].isin(arah_angin_valid)]
        if not arah_angin_tidak_valid.empty:
            hasil_tidak_konsisten['Arah Angin Tidak Valid'] = arah_angin_tidak_valid[['wd']]
            print("\nDitemukan arah angin yang tidak valid:")
            print(arah_angin_tidak_valid[['wd']])

    # 3. Memeriksa inkonsistensi dalam suhu dan titik embun
    if 'TEMP' in df.columns and 'DEWP' in df.columns:
        suhu_tidak_konsisten = df[df['DEWP'] > df['TEMP']]
        if not suhu_tidak_konsisten.empty:
            hasil_tidak_konsisten['DEWP > TEMP'] = suhu_tidak_konsisten[['TEMP', 'DEWP']]
            print("\nDitemukan titik embun yang lebih tinggi dari suhu:")
            print(suhu_tidak_konsisten[['TEMP', 'DEWP']])

    # 4. Memeriksa inkonsistensi dalam kecepatan angin
    if 'WSPM' in df.columns:
        kecepatan_angin_negatif = df[df['WSPM'] < 0]
        if not kecepatan_angin_negatif.empty:
            hasil_tidak_konsisten['Kecepatan Angin Negatif'] = kecepatan_angin_negatif[['WSPM']]
            print("\nDitemukan kecepatan angin negatif:")
            print(kecepatan_angin_negatif[['WSPM']])

    if not hasil_tidak_konsisten:
        print("Tidak ditemukan nilai yang tidak konsisten berdasarkan kriteria yang diperiksa.")

    return hasil_tidak_konsisten

# Menjalankan fungsi
hasil_tidak_konsisten = cek_nilai_tidak_konsisten(aotizhongxin_df)

Pemeriksaan nilai tidak konsisten:

Ditemukan nilai PM2.5 yang lebih besar dari PM10:
       PM2.5   PM10
273    116.0   90.0
274    123.0   94.0
275    130.0  104.0
276    132.0   87.0
277    129.0   84.0
...      ...    ...
34011   93.0   90.0
34013   97.0   95.0
34025  122.0  113.0
34026  128.0  122.0
34049  155.0  151.0

[1183 rows x 2 columns]

Ditemukan arah angin yang tidak valid:
        wd
6388   NaN
11718  NaN
13412  NaN
16748  NaN
17263  NaN
...    ...
34314  NaN
34334  NaN
34560  NaN
34638  NaN
34746  NaN

[81 rows x 1 columns]


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def cek_outlier(df):
    print("Pemeriksaan outlier menggunakan metode IQR:")

    # Pilih kolom numerik
    kolom_numerik = df.select_dtypes(include=[np.number]).columns

    hasil_outlier = {}

    for kolom in kolom_numerik:
        Q1 = df[kolom].quantile(0.25)
        Q3 = df[kolom].quantile(0.75)
        IQR = Q3 - Q1

        batas_bawah = Q1 - 1.5 * IQR
        batas_atas = Q3 + 1.5 * IQR

        outlier = df[(df[kolom] < batas_bawah) | (df[kolom] > batas_atas)]

        if not outlier.empty:
            hasil_outlier[kolom] = outlier[[kolom]]
            print(f"\nOutlier ditemukan di kolom {kolom}:")
            print(f"Jumlah outlier: {len(outlier)}")
            print(f"Persentase outlier: {(len(outlier) / len(df)) * 100:.2f}%")
            print(f"Range nilai outlier: {outlier[kolom].min()} hingga {outlier[kolom].max()}")


    if not hasil_outlier:
        print("Tidak ditemukan outlier berdasarkan metode IQR.")

    return hasil_outlier

# Menjalankan fungsi
hasil_outlier = cek_outlier(aotizhongxin_df)

Pemeriksaan outlier menggunakan metode IQR:

Outlier ditemukan di kolom PM2.5:
Jumlah outlier: 1624
Persentase outlier: 4.63%
Range nilai outlier: 253.0 hingga 898.0

Outlier ditemukan di kolom PM10:
Jumlah outlier: 1080
Persentase outlier: 3.08%
Range nilai outlier: 331.0 hingga 984.0

Outlier ditemukan di kolom SO2:
Jumlah outlier: 3054
Persentase outlier: 8.71%
Range nilai outlier: 49.0 hingga 341.0

Outlier ditemukan di kolom NO2:
Jumlah outlier: 472
Persentase outlier: 1.35%
Range nilai outlier: 161.0 hingga 290.0

Outlier ditemukan di kolom CO:
Jumlah outlier: 2607
Persentase outlier: 7.43%
Range nilai outlier: 3100.0 hingga 10000.0

Outlier ditemukan di kolom O3:
Jumlah outlier: 1310
Persentase outlier: 3.74%
Range nilai outlier: 193.851 hingga 423.0

Outlier ditemukan di kolom RAIN:
Jumlah outlier: 1380
Persentase outlier: 3.94%
Range nilai outlier: 0.1 hingga 72.5

Outlier ditemukan di kolom WSPM:
Jumlah outlier: 1742
Persentase outlier: 4.97%
Range nilai outlier: 4.2 hingga 1

In [ ]:


# Buat tabel markdown
table = """
| Dataset     | Tipe data                                    | Missing value                                     | Duplicate data                     | Inaccurate value                       |
|-------------|----------------------------------------------|---------------------------------------------------|-------------------------------------|----------------------------------------|
| aotizhongxin | <br>Terdapat kesalahan tipe :<br>1.data(hour,day,month,year)menjadi date_times<br><br>2.colom wd menjadi categori <br>3. station menjadi categori<br>| Terdapat   missing values:    <br> 1. 925 missing values pada PM2.5. <br>    | Terdapat 11 data yang duplikat.      | Terdapat inaccurate value pada kolom age. |
| orders_df   | Terdapat kesalahan tipe data untuk kolom order_date & delivery_date. | - | -                                   | -                                      |
| product_df  | -                                            | -                                                 | Terdapat 6 data yang duplikat.      | -                                      |
| sales_df    | -                                            | Terdapat 19 missing value pada kolom total_price. | -                                   | -                                      |
"""




In [ ]:
display(Markdown(table))



| Dataset     | Tipe data                                    | Missing value                                     | Duplicate data                     | Inaccurate value                       |
|-------------|----------------------------------------------|---------------------------------------------------|-------------------------------------|----------------------------------------|
| aotizhongxin | <br>Terdapat kesalahan tipe :<br>1.data(hour,day,month,year)menjadi date_times<br><br>2.colom wd menjadi categori <br>3. station menjadi categori<br>| Terdapat   missing values:    <br> 1. 925 missing values pada PM2.5. <br>    | Terdapat 11 data yang duplikat.      | Terdapat inaccurate value pada kolom age. |
| orders_df   | Terdapat kesalahan tipe data untuk kolom order_date & delivery_date. | - | -                                   | -                                      |
| product_df  | -                                            | -                                                 | Terdapat 6 data yang duplikat.      | -                                      |
| sales_df    | -                                            | Terdapat 19 missing value pada kolom total_price. | -                                   | -                                      |


## 6 Cleaning Data

### 6.1 Aotizhongxin

#### 6.2 Merubah tipe data

In [9]:
# Menggabungkan kolom year, month, day, dan hour menjadi satu kolom datetime
aotizhongxin_df['datetime'] = pd.to_datetime(aotizhongxin_df[['year', 'month', 'day', 'hour']])

In [10]:
aotizhongxin_df.set_index('datetime', inplace=True)

In [11]:
aotizhongxin_df['wd'] = aotizhongxin_df['wd'].astype('category')

In [12]:
aotizhongxin_df['station'] = aotizhongxin_df['station'].astype('category')

#### 6.1.3 Mengisi nilai yang hilang

In [13]:
pollutants = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3']
aotizhongxin_df[pollutants] = aotizhongxin_df[pollutants].interpolate(method='time')

In [14]:
meteorological = ['TEMP', 'PRES', 'DEWP', 'RAIN']
aotizhongxin_df[meteorological] = aotizhongxin_df[meteorological].interpolate(method='linear')

In [15]:
aotizhongxin_df['wd'] = aotizhongxin_df['wd'].fillna(method='ffill')

<ipython-input-15-5663bc40dd4c>:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  aotizhongxin_df['wd'] = aotizhongxin_df['wd'].fillna(method='ffill')


In [17]:
aotizhongxin_df['WSPM'] = aotizhongxin_df['WSPM'].fillna(method='ffill')

<ipython-input-17-d375f5bb1d31>:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  aotizhongxin_df['WSPM'] = aotizhongxin_df['WSPM'].fillna(method='ffill')


In [18]:
aotizhongxin_df.isna().sum()

,0
No,0
year,0
month,0
day,0
hour,0
PM2.5,0
PM10,0
SO2,0
NO2,0
CO,0


In [19]:
aotizhongxin_df.head(5)

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
datetime,,,,,,,,,,,,,,,,,,
2013-03-01 00:00:00,1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
2013-03-01 01:00:00,2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin
2013-03-01 02:00:00,3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin
2013-03-01 03:00:00,4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin
2013-03-01 04:00:00,5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin


### 6.2 changping

#### merubah tipe data

In [20]:
changping_df['datetime'] = pd.to_datetime(changping_df[['year', 'month', 'day', 'hour']])

In [21]:
changping_df.set_index('datetime', inplace=True)

In [22]:
changping_df['wd'] = changping_df['wd'].astype('category')
changping_df['station'] = changping_df['station'].astype('category')
changping_df['PRES'] = changping_df['PRES'].astype('float64')
changping_df['DEWP'] = changping_df['DEWP'].astype('float64')

In [27]:
pollutants = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3']
changping_df[pollutants] = changping_df[pollutants].interpolate(method='time')

In [28]:
meteorological = ['TEMP', 'PRES', 'DEWP', 'RAIN']
changping_df[meteorological] = changping_df[meteorological].interpolate(method='linear')

## 7 Exploratory Data Analysis (EDA)

### 7.1 Explore

#### 7.1.1 Aoti

In [ ]:
# Fungsi untuk menjalankan query SQL
pysqldf = lambda q: sqldf(q, globals())

In [36]:
query = '''
SELECT
    year,
    month,
    day,
    hour,
    AVG("PM2.5") as avg_pm25,
    AVG("PM10") as avg_pm10,
    AVG("SO2") as avg_so2,
    AVG("NO2") as avg_no2,
    AVG("CO") as avg_co,
    AVG("O3") as avg_o3
FROM
    aotizhongxin_df
WHERE
    year = 2013 AND month = 3 AND day = 1 AND hour BETWEEN 0 AND 19
GROUP BY
    year, month, day, hour
ORDER BY
    year, month, day, hour;
'''

# Eksekusi query
partikulasi_polusi_harian = psql.sqldf(query, globals())

# Membuat kolom `date_time` dari `year`, `month`, `day`, dan `hour`
partikulasi_polusi_harian['date_time'] = pd.to_datetime(partikulasi_polusi_harian[['year', 'month', 'day', 'hour']])

# Menampilkan hasil dengan kolom `date_time`
partikulasi_polusi_harian[['date_time', 'avg_pm25', 'avg_pm10', 'avg_so2', 'avg_no2', 'avg_co', 'avg_o3']]


,date_time,avg_pm25,avg_pm10,avg_so2,avg_no2,avg_co,avg_o3
0,2013-03-01 00:00:00,4.0,4.0,4.0,7.0,300.0,77.0
1,2013-03-01 01:00:00,8.0,8.0,4.0,7.0,300.0,77.0
2,2013-03-01 02:00:00,7.0,7.0,5.0,10.0,300.0,73.0
3,2013-03-01 03:00:00,6.0,6.0,11.0,11.0,300.0,72.0
4,2013-03-01 04:00:00,3.0,3.0,12.0,12.0,300.0,72.0
5,2013-03-01 05:00:00,5.0,5.0,18.0,18.0,400.0,66.0
6,2013-03-01 06:00:00,3.0,3.0,18.0,32.0,500.0,50.0
7,2013-03-01 07:00:00,3.0,6.0,19.0,41.0,500.0,43.0
8,2013-03-01 08:00:00,3.0,6.0,16.0,43.0,500.0,45.0
9,2013-03-01 09:00:00,3.0,8.0,12.0,28.0,400.0,59.0


In [42]:
query = '''
SELECT
    year,
    month,
    day,
    ROUND(AVG("PM2.5")) as avg_PM25,
    ROUND(AVG("PM10")) as avg_PM10,
    ROUND(AVG("SO2")) as avg_SO2,
    ROUND(AVG("NO2")) as avg_NO2,
    ROUND(AVG("CO")) as avg_CO,
    ROUND(AVG("O3")) as avg_O3
FROM
    aotizhongxin_df
WHERE
    (year = 2015 AND month = 1 AND day BETWEEN 1 AND 10)
GROUP BY
    year, month, day
ORDER BY
    year, month, day;
'''
partikulasi_polusi_harian = psql.sqldf(query, globals())

# Membuat kolom 'date_time' dari kolom 'year', 'month', 'day'
partikulasi_polusi_harian['date_time'] = pd.to_datetime(partikulasi_polusi_harian[['year', 'month', 'day']])

# Menampilkan hasil dengan kolom date_time
partikulasi_polusi_harian[['date_time', 'avg_PM25', 'avg_PM10', 'avg_SO2', 'avg_NO2', 'avg_CO', 'avg_O3']]


,date_time,avg_PM25,avg_PM10,avg_SO2,avg_NO2,avg_CO,avg_O3
0,2015-01-01,38.0,61.0,22.0,49.0,942.0,24.0
1,2015-01-02,59.0,84.0,34.0,69.0,1338.0,14.0
2,2015-01-03,161.0,196.0,77.0,117.0,3558.0,2.0
3,2015-01-04,190.0,222.0,83.0,128.0,4217.0,4.0
4,2015-01-05,101.0,162.0,62.0,68.0,2892.0,27.0
5,2015-01-06,29.0,55.0,24.0,38.0,838.0,25.0
6,2015-01-07,53.0,76.0,37.0,66.0,1613.0,7.0
7,2015-01-08,130.0,173.0,54.0,103.0,3321.0,2.0
8,2015-01-09,76.0,98.0,44.0,69.0,1879.0,22.0
9,2015-01-10,132.0,166.0,58.0,103.0,2892.0,10.0


In [45]:
query = '''
SELECT
    year,
    month,
    ROUND(AVG("PM2.5")) as avg_PM25,
    ROUND(AVG("PM10")) as avg_PM10,
    ROUND(AVG("SO2")) as avg_SO2,
    ROUND(AVG("NO2")) as avg_NO2,
    ROUND(AVG("CO")) as avg_CO,
    ROUND(AVG("O3")) as avg_O3
FROM
    aotizhongxin
WHERE
    (year = 2015 AND month BETWEEN 1 AND 10)
GROUP BY
    year, month
ORDER BY
    year, month;
'''
partikulasi_polusi_bulanan = psql.sqldf(query, globals())

partikulasi_polusi_harian['date_time'] = pd.to_datetime(partikulasi_polusi_harian['year', 'month'])

partikulasi_polusi_harian[['date_time', 'avg_PM25', 'avg_PM10', 'avg_SO2', 'avg_NO2', 'avg_CO', 'avg_O3']]


PandaSQLException: (sqlite3.OperationalError) no such table: aotizhongxin
[SQL: 
SELECT 
    year, 
    month, 
    ROUND(AVG("PM2.5")) as avg_PM25,
    ROUND(AVG("PM10")) as avg_PM10,
    ROUND(AVG("SO2")) as avg_SO2,
    ROUND(AVG("NO2")) as avg_NO2,
    ROUND(AVG("CO")) as avg_CO,
    ROUND(AVG("O3")) as avg_O3
FROM 
    aotizhongxin
WHERE 
    (year = 2015 AND month BETWEEN 1 AND 10)
GROUP BY 
    year, month
ORDER BY 
    year, month;
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [47]:
# Assuming your dataframe is named aotizhongxin_df
query = '''
SELECT
    year,
    month,
    ROUND(AVG("PM2.5")) as avg_PM25,
    ROUND(AVG("PM10")) as avg_PM10,
    ROUND(AVG("SO2")) as avg_SO2,
    ROUND(AVG("NO2")) as avg_NO2,
    ROUND(AVG("CO")) as avg_CO,
    ROUND(AVG("O3")) as avg_O3
FROM
    aotizhongxin_df
WHERE
    (year = 2015 AND month BETWEEN 1 AND 10)
GROUP BY
    year, month
ORDER BY
    year, month;
'''
partikulasi_polusi_bulanan = psql.sqldf(query, globals())

# Assuming you want to use partikulasi_polusi_bulanan for the datetime column
partikulasi_polusi_bulanan['date_time'] = pd.to_datetime(partikulasi_polusi_bulanan[['year', 'month']].assign(day=1)) # Added day=1 for to_datetime

partikulasi_polusi_bulanan[['date_time', 'avg_PM25', 'avg_PM10', 'avg_SO2', 'avg_NO2', 'avg_CO', 'avg_O3']]

,date_time,avg_PM25,avg_PM10,avg_SO2,avg_NO2,avg_CO,avg_O3
0,2015-01-01,91.0,115.0,38.0,69.0,2047.0,19.0
1,2015-02-01,85.0,123.0,30.0,57.0,1506.0,46.0
2,2015-03-01,88.0,156.0,23.0,61.0,1250.0,54.0
3,2015-04-01,77.0,135.0,10.0,55.0,817.0,77.0
4,2015-05-01,59.0,110.0,9.0,48.0,763.0,101.0
5,2015-06-01,62.0,86.0,7.0,45.0,1047.0,102.0
6,2015-07-01,66.0,84.0,6.0,52.0,844.0,121.0
7,2015-08-01,47.0,71.0,4.0,43.0,870.0,97.0
8,2015-09-01,52.0,72.0,6.0,60.0,990.0,64.0
9,2015-10-01,78.0,97.0,6.0,72.0,1182.0,41.0


## 8 Visualization & Explanatory Analysis

### 8.1 Pertanyaan 1:

### 8.1 Pertanyaan 2:

## 9 Analisis Lanjutan (Opsional)

## 10 Conclusion